In [16]:
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings # import hf embedding
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import re
import pandas as pd
from datetime import date, timedelta
import datetime
import os
from pinecone import Pinecone
import time
import dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
import uuid
from tqdm import tqdm
dotenv.load_dotenv()

True

In [233]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")

OAI_embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

In [234]:

start_date = date.today()
#start_date = datetime.datetime(2024, 7, 31)

end_date = start_date + timedelta(days=150)  # For example, 30 days from today

# List to hold all dates between start and end dates
date_list = []

# Generate the list of dates
current_date = start_date
while current_date <= end_date:
    date_list.append(current_date)
    current_date += timedelta(days=1)

# Optionally, print the dates with year, month, and day
# for date in date_list:
#     print(date.year, date.month, date.day)

len(date_list)


151

In [236]:
loader = WebBaseLoader("https://sf.funcheap.com/2024/05/03/")
data = loader.load()
len(data[0].page_content)

3835

In [237]:
data[0].page_content\
    .replace('-','')\
    .replace(' – ','')\
    .replace('-','')\
    .replace('Log in','')\
    .replace('Do Not Sell My Personal Information','')\
    .replace('Subscribe','')\
    .replace('Follow Us','')\
    .split('Search for:')[-1]\
    .strip().split('Proudly powered by WordPress')[0]

"SF’s May Festivals & Fairs\n\n\nSF’s New 5Acre Waterfront Park Opens\n\n\nRoller Skate Rentals Return to GG Park \n\n\nSF’s Japantown is Getting a Huge Makeover\n\n\nFree Museum Days\n\n\nThis Weekend’s Events (May 35)\n\n\n\n \n\n\n\n\nToday\nWeekend\nSelect Date\nTop Picks\n\n\n\n\n\n\t\t\t\t\tEvents for May 03, 2024\t\t\t\t\n\n\n< Thu. May 2\nSat. May 4 >\n\n\nMost Popular Events\nSan Leandro’s 1st Annual “MexiPino Fiesta” (2024)“The Fillmore Eclipse” SF’s New Immersive Theater Experience (April 26May 12)North Beach First Fridays Art Crawl (SF)Japanese Tea Garden’s Free Admission Hour (Golden Gate Park)“Star Wars: The Empire Strikes Back” Film w/ Live Orchestra (May 24)Friday, May 3> See all\n\n\n\n\n“Crazy Funny Asians” Friday Night Comedy Show (SF) Every FridaySponsored  7:00 pm  Ends at 10:30 pm\nFREE*\nBayview / Hunters Point, San Francisco\n\n\n\n\n\n\n\nSF’s New Star Wars Bar Hosts PopUp Comedy (May 25) Sponsored  7:00 pm  Ends at 10:30 pm\nFREE*\nPolk Gulch, Tenderloin, San 

## pull from single date

In [37]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=650,
    chunk_overlap=35, 
    length_function = len,
    separators=["\n\n","\n\t\t\n\n*","\n"," "]
)

In [172]:
event_dates = {'date':[],'link':[],'content':[]}

all_splits = []
events_docs = None
for date in date_list:
    event_dates['date'].append(date)
    link = f"https://sf.funcheap.com/{date.year}/{date.month}/{date.day}/"
    event_dates['link'].append(link)
    loader = WebBaseLoader(link)
    data = loader.load()
    
    # day_before = date - datetime.timedelta(days=1)
    # day_after = date + datetime.timedelta(days=1)
    # formatted_day_before = day_before.strftime("%A, %B %-d")  # Saturday, May 4
    # formatted_day_after = day_after.strftime("%A, %B %-d")    # Monday, May 6
    # formatted_given_date = date.strftime("%A, %B %-d")  # Sunday, May 5
    # formatted_date_tag = f"< {formatted_day_before}\n{formatted_day_after} >\n\n{formatted_given_date}"

    #events_text = data[0].page_content.split(formatted_date_tag)[-1].split(formatted_day_after)[0]
    
    events_text = data[0].page_content\
    .replace('-','')\
    .replace(' – ','')\
    .replace('-','')\
    .replace('Log in','')\
    .replace('Do Not Sell My Personal Information','')\
    .replace('Subscribe','')\
    .replace('Follow Us','')\
    .split('Search for:')[-1]\
    .strip().split('Proudly powered by WordPress')[0]
    
    splits = r_splitter.split_text(events_text)
    event_dates['content'].append(splits)
    all_splits.append(splits)
    docs = r_splitter.create_documents(splits)
    if events_docs is None:
        events_docs = docs
    else:
        events_docs += docs
    print(f'finished scraping {date}: {len(events_docs)}')


finished scraping 2024-05-01: 20
finished scraping 2024-05-02: 39
finished scraping 2024-05-03: 72
finished scraping 2024-05-04: 106
finished scraping 2024-05-05: 140
finished scraping 2024-05-06: 150
finished scraping 2024-05-07: 166
finished scraping 2024-05-08: 182
finished scraping 2024-05-09: 197
finished scraping 2024-05-10: 219
finished scraping 2024-05-11: 247
finished scraping 2024-05-12: 272
finished scraping 2024-05-13: 281
finished scraping 2024-05-14: 293
finished scraping 2024-05-15: 308
finished scraping 2024-05-16: 321
finished scraping 2024-05-17: 342
finished scraping 2024-05-18: 370
finished scraping 2024-05-19: 390
finished scraping 2024-05-20: 399
finished scraping 2024-05-21: 409
finished scraping 2024-05-22: 427
finished scraping 2024-05-23: 441
finished scraping 2024-05-24: 458
finished scraping 2024-05-25: 482
finished scraping 2024-05-26: 500
finished scraping 2024-05-27: 509
finished scraping 2024-05-28: 519
finished scraping 2024-05-29: 532
finished scraping

In [213]:
event_dates_df = pd.DataFrame(event_dates)
event_dates_df = event_dates_df.explode('content')
event_dates_df['id'] = [str(uuid.uuid4()) for i in range(event_dates_df.shape[0])]
event_dates_df

,date,link,content,id
0,2024-05-01,https://sf.funcheap.com/2024/5/1/,"Events for May 01, 2024 Funcheap\n \n\n\n\n\n...",59809655-7a4e-4ce2-ae12-a42cdc063b93
0,2024-05-01,https://sf.funcheap.com/2024/5/1/,Join the Funcheap email list\r\n\tJoin the 140...,0a5cbe3e-cc96-4ca0-ac49-4ba262ef1461
0,2024-05-01,https://sf.funcheap.com/2024/5/1/,Most Popular Events\nJapanese Tea Garden’s Fre...,690bc4ee-157a-4841-885a-2534cbb70ac3
0,2024-05-01,https://sf.funcheap.com/2024/5/1/,| Museum of Craft and Design \nThere’s no bett...,1bb7f628-af2c-4cd2-8546-00f5bbdc7344
0,2024-05-01,https://sf.funcheap.com/2024/5/1/,| KitTea Cat Lounge \nSan Francisco’s only Cat...,0e8dd8a4-854b-4173-9109-48041763bfff
...,...,...,...,...
150,2024-09-28,https://sf.funcheap.com/2024/9/28/,Most Popular Upcoming Events\n\n\n\n\n\n ...,96711c7f-1787-48fd-89f2-181198a22089
150,2024-09-28,https://sf.funcheap.com/2024/9/28/,2024 Taiwanese American Cultural Festival (Uni...,5b14dded-23fb-4bf4-bfc0-31943bd09a90
150,2024-09-28,https://sf.funcheap.com/2024/9/28/,Most Popular City Guides\n\n\n\n\n\n ...,59358757-c1e9-48a7-8a9b-f521cca485fb
150,2024-09-28,https://sf.funcheap.com/2024/9/28/,SF’s “Chinatown Night Market” Returns for 2024...,efe61bdf-7d05-43a8-b32b-8ca2a460a256


In [214]:
event_dates_df['id'].nunique()

1672

In [178]:
import itertools
combined_splits = list(itertools.chain.from_iterable(all_splits))

In [179]:
len(all_splits),len(combined_splits)

(151, 1672)

# FAISS

In [129]:
db = FAISS.from_documents(events_docs, embeddings)
db.save_local(f"funcheap_{str(date_list[0])}_{str(date_list[0-1])}_db")

In [226]:
def load_embeddings():
    return HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', model_kwargs={'device': 'cpu'})

def load_RAG_db():
    vectordb = FAISS.load_local("../vector_db/funcheap_2024-04-26_2024-06-25_db",
                      load_embeddings())
    return vectordb

llm = Ollama(model='mistral')
db = load_RAG_db()

chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=db.as_retriever(),return_source_documents=True,
                                              rephrase_question=True)

In [228]:
# query="""visiting the bay area in may with my family. we prefer sightseeing and would like to see the golden gate bridge. are there any events happening that are good for
# children? My toddler is developing an interest for dinosaurs, perhaps a recommendation for museums?"""
result = chain({"question": query,"chat_history": []})

In [229]:
result

{'question': 'when is the 2024 Taiwanese American Cultural Festival',
 'chat_history': [],
 'answer': ' The text provided does not mention a "Taiwanese American Cultural Festival" taking place on May 10, 2024. The text mentions various other events occurring on that date, but none of them are described as being specifically Taiwanese American in nature.',
 'source_documents': [Document(page_content='| Cityview Event Hall (Treasure Island) \nMark your calendars for May 10, 2024 as we come together to celebrate the 2024 Asian American and Pacific Islander Mental Health Day!\nJoin us for engaging activities such as Drumming Circle, Siva4Wellness, Hula, Southeast Asian Dance, CommuniTEA, and more! Come discover RAMS’ 50 years of ...\n\nSF’s Friday Night Market “Bhangra & Beats” (2024)\nFriday, May 10 – 5:00 pm  - Ends at 10:00 pm |'),
  Document(page_content='Cost: FREE* \n\t\t\n\n*Free for all ages\n\n | Japanese Cultural and Community Center of Northern California \nGet the kids together

In [44]:
sample_questions = ["From April 30th to May 15th, as a solo traveler who values quiet introspection and serene landscapes, can you suggest hidden coastal retreats or secluded hiking trails in the Bay Area where I can reconnect with nature and find inner peace?",
"Between May 8th and June 10th, being a passionate epicurean with a weakness for artisanal delights and off-the-beaten-path culinary experiences, I'd love your recommendations for quaint family-owned eateries, tucked-away cafes, or vibrant food festivals where I can indulge in the authentic flavors of the Bay Area.",
"During the week of May 20th to May 27th, immersing myself in local culture and heritage is paramount to me. Could you share insider tips on lesser-known cultural festivals, neighborhood walking tours led by community storytellers, or underground art collectives that offer a glimpse into the soul of San Francisco?",
"From June 5th to June 18th, as an adrenaline junkie constantly seeking the next thrill, I'm eager to discover adrenaline-pumping activities unique to the Bay Area. From adrenaline-fueled kayaking expeditions under the Golden Gate Bridge to daredevil cliff diving spots along the coast, what are some lesser-known adventures that will get my heart racing?",
"Between April 28th and June 25th, with a profound appreciation for the arts and a desire to uncover hidden artistic gems, I'm on the lookout for intimate gallery openings, eclectic street art alleys, or underground poetry slams where local creatives converge. Can you provide insider insights into the vibrant artistic subculture thriving within the Bay Area?"]

for q in sample_questions:
    print(q)
    result = chain({"question": q,"chat_history": []})
    print(result['answer'])
    print("*" * 15)
    

From April 30th to May 15th, as a solo traveler who values quiet introspection and serene landscapes, can you suggest hidden coastal retreats or secluded hiking trails in the Bay Area where I can reconnect with nature and find inner peace?
 Based on the context provided, there isn't any specific mention of hidden coastal retreats or secluded hiking trails from April 30th to May 15th in the Bay Area. However, I can suggest some beautiful and serene places that you might find interesting for your solo travel:

1. Point Reyes National Seashore - Located north of San Francisco, this park offers stunning coastal views, peaceful beaches, and various hiking trails with breathtaking scenery. It's an excellent place to reconnect with nature and find inner peace.
2. Muir Woods National Monument - Another beautiful natural retreat located just north of San Francisco. Muir Woods is a redwood forest with lush greenery, peaceful streams, and quiet trails that are perfect for introspection and relaxa

# Pinecone

In [34]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY") )

In [244]:
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

In [218]:
index_name = 'wanderchat-funcheap-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='dotproduct',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [219]:
from tqdm.auto import tqdm  # for progress bar

data = event_dates_df  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"i-{x['id']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['content'] for _, x in batch.iterrows()]
    # embed text
    embeds = OAI_embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {
         'text':x['content'],
         'source': x['link'],
         'date': x['date']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

100%|██████████| 17/17 [01:17<00:00,  4.55s/it]


In [220]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1672}},
 'total_vector_count': 1672}

In [240]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, OAI_embed_model.embed_query, text_field
)

/Users/christopherton/Documents/WanderChat_298b/venv/lib/python3.10/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [239]:
query = "when is the 2024 Taiwanese American Cultural Festival"

vectorstore.similarity_search(query, k=3)

[Document(page_content='2024 Taiwanese American Cultural Festival (Union Square)                    \n\n\n\n                        2024 San Francisco Carnaval Grand Parade                    \n\n\n\n                        SF’s New Monthly Block Party “Downtown First Thursdays” KickOff                    \n\n\n\n                        San Francisco’s World Goth Day Festival 2024 on USS Hornet (Alameda)                    \n\n\n\nSign Up For Funcheap Emails \n\n\n\n\nJoin the Funcheap SF Email List\n\n\n\n\r\nJoin the 140,000+ San Franciscans and get our picks for the best  Bay Area free & cheap events and deals each week.\r\n\r\n\r\n\n\nMost Popular City Guides', metadata={'date': '2024-05-18', 'source': 'https://sf.funcheap.com/2024/5/18/'}),
 Document(page_content='2024 Taiwanese American Cultural Festival (Union Square)                    \n\n\n\n                        2024 San Francisco Carnaval Grand Parade                    \n\n\n\n                        SF’s New Monthly Blo

In [230]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    #print(augmented_prompt)
    return augmented_prompt

In [231]:
p = augment_prompt(query)
p

'Using the contexts below, answer the query.\n\n    Contexts:\n    2024 Taiwanese American Cultural Festival (Union Square)                    \n\n\n\n                        2024 San Francisco Carnaval Grand Parade                    \n\n\n\n                        SF’s New Monthly Block Party “Downtown First Thursdays” KickOff                    \n\n\n\n                        San Francisco’s World Goth Day Festival 2024 on USS Hornet (Alameda)                    \n\n\n\nSign Up For Funcheap Emails \n\n\n\n\nJoin the Funcheap SF Email List\n\n\n\n\r\nJoin the 140,000+ San Franciscans and get our picks for the best  Bay Area free & cheap events and deals each week.\r\n\r\n\r\n\n\nMost Popular City Guides\n2024 Taiwanese American Cultural Festival (Union Square)                    \n\n\n\n                        2024 San Francisco Carnaval Grand Parade                    \n\n\n\n                        SF’s New Monthly Block Party “Downtown First Thursdays” KickOff                    \

In [225]:
llm.invoke(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    2024 Taiwanese American Cultural Festival (Union Square)                    



                        2024 San Francisco Carnaval Grand Parade                    



                        SF’s New Monthly Block Party “Downtown First Thursdays” KickOff                    



                        San Francisco’s World Goth Day Festival 2024 on USS Hornet (Alameda)                    



Sign Up For Funcheap Emails 




Join the Funcheap SF Email List




Join the 140,000+ San Franciscans and get our picks for the best  Bay Area free & cheap events and deals each week.




Most Popular City Guides
2024 Taiwanese American Cultural Festival (Union Square)                    



                        2024 San Francisco Carnaval Grand Parade                    



                        SF’s New Monthly Block Party “Downtown First Thursdays” KickOff                    



                        San Francisco’s World Got

" The 2024 Taiwanese American Cultural Festival is scheduled to take place in Union Square. Exact dates for the event have not been specified in the context provided. To get the most accurate and up-to-date information, it's recommended to sign up for Funcheap emails or check their website regularly for announcements regarding the festival date."

In [203]:
llm.invoke(sample_questions[0])

' Absolutely! Here are some hidden gems in the Bay Area that offer beautiful coastal scenery and quiet hiking trails for introspective solo travelers:\n\n1. Samuel P. Taylor State Park, Marin County: This park is located along the coast of Marin County and features a network of hiking trails through redwood forests and open grasslands. The park also has two secluded beaches where you can enjoy the peaceful sounds of the waves. The Rodeo Lagoon Trail and the Dipsea Trail are two popular hikes that offer scenic views and tranquility.\n\n2. Fitzgerald Marine Reserve, Moss Landing: This marine reserve is a hidden gem located in Monterey Bay. It offers a quiet and serene environment for solo travelers seeking introspection and connection with nature. The park has several hiking trails that lead to scenic viewpoints overlooking the bay and the ocean. You can also explore the tide pools at low tide for a closer look at marine life.\n\n3. Point Reyes National Seashore, Marin County: This seclu

# Travel Advisory

In [241]:
#https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories/israel-west-bank-and-gaza-travel-advisory.html
#https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/

In [274]:
urls = [
    'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories/israel-west-bank-and-gaza-travel-advisory.html',
    'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/'
]

docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=50
)
doc_splits = text_splitter.split_documents(docs_list)

In [283]:
doc_splits[30].metadata

{'source': 'https://travel.state.gov/content/travel/en/traveladvisories/traveladvisories.html/',
 'title': 'Travel Advisories'}

In [292]:
travel_advisory_df = pd.DataFrame(columns=['page_content','source','title'])

for d in range(len(doc_splits)):
    travel_advisory_df.loc[d,'page_content'] = doc_splits[d].page_content
    travel_advisory_df.loc[d,'source'] = doc_splits[d].metadata['source']
    travel_advisory_df.loc[d,'title'] = doc_splits[d].metadata['title']

travel_advisory_df = travel_advisory_df.reset_index()
travel_advisory_df.head()

,index,page_content,source,title
0,0,"Israel, the West Bank and Gaza Travel Advisory...",https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
1,1,Popular Links\n\nHome\nTravel Advisories\n New...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
2,2,International Parental Child Abduction\n\n\n\n...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
3,3,G\nH\nI\nJ\nK\nL\n\n\n\n\nM\nN\nO\nP\nQ\nR\n\n...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
4,4,Gaza due to terrorism and armed conflict\n\nRe...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"


,level_0,index,page_content,source,title
0,0,0,"Israel, the West Bank and Gaza Travel Advisory...",https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
1,1,1,Popular Links\n\nHome\nTravel Advisories\n New...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
2,2,2,International Parental Child Abduction\n\n\n\n...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
3,3,3,G\nH\nI\nJ\nK\nL\n\n\n\n\nM\nN\nO\nP\nQ\nR\n\n...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"
4,4,4,Gaza due to terrorism and armed conflict\n\nRe...,https://travel.state.gov/content/travel/en/tra...,"Israel, the West Bank and Gaza Travel Advisory"


In [331]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

In [263]:
index_name = 'wanderchat-travel-advisory-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [35]:
embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")

/Users/christopherton/Documents/WanderChat_298b/venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [295]:
from tqdm.auto import tqdm  # for progress bar

data = travel_advisory_df  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    i_end = min(len(data), i+batch_size)
    # get batch of data
    batch = data.iloc[i:i_end]
    # generate unique ids for each chunk
    ids = [f"{x['index']}" for i, x in batch.iterrows()]
    # get text to embed
    texts = [x['page_content'] for _, x in batch.iterrows()]
    # embed text
    embeds = embed_model.embed_documents(texts)
    # get metadata to store in Pinecone
    metadata = [
        {'text': x['page_content'],
         'source': x['source'],
         'title': x['title']} for i, x in batch.iterrows()
    ]
    # add to Pinecone
    index.upsert(vectors=zip(ids, embeds, metadata))

100%|██████████| 1/1 [00:02<00:00,  2.62s/it]


In [296]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 43}},
 'total_vector_count': 43}

In [297]:
from langchain.vectorstores import Pinecone

text_field = "text"  # the metadata field that contains our text

# initialize the vector store object
vectorstore = Pinecone(
    index, embed_model.embed_query, text_field
)

/Users/christopherton/Documents/WanderChat_298b/venv/lib/python3.10/site-packages/langchain_community/vectorstores/pinecone.py:75: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [299]:
query = "Is it safe to travel to Gaza?"

vectorstore.similarity_search(query, k=3)

[Document(page_content='Gaza – Do Not Travel\nDo not travel due to\xa0terrorism and\xa0armed conflict.\nThe U.S. government is unable to provide routine or emergency consular services to U.S. citizens in Gaza as U.S. government employees are prohibited from traveling there. The Israel Defense Forces (IDF) are conducting large-scale military operations in Gaza against Hamas, a U.S. government-designated foreign terrorist organization, which was responsible for the October 7 attack on Israel. As a result of the armed conflict, the security environment within Gaza and on its borders is extremely dangerous and volatile. The pedestrian crossing between Gaza and Israel was damaged on October 7 and remains closed, and the pedestrian crossing between Egypt and Gaza may close without advance notice depending on the security situation. There are sporadic telecommunication and internet outages within Gaza further inhibiting the ability of residents to obtain information.\xa0\nVisit our website fo

In [300]:
def augment_prompt(query: str):
    # get top 3 results from knowledge base
    results = vectorstore.similarity_search(query, k=3)
    # get the text from the results
    source_knowledge = "\n".join([x.page_content for x in results])
    # feed into an augmented prompt
    augmented_prompt = f"""Using the contexts below, answer the query.

    Contexts:
    {source_knowledge}

    Query: {query}"""
    return augmented_prompt

In [301]:
print(augment_prompt(query))

Using the contexts below, answer the query.

    Contexts:
    Gaza – Do Not Travel
Do not travel due to terrorism and armed conflict.
The U.S. government is unable to provide routine or emergency consular services to U.S. citizens in Gaza as U.S. government employees are prohibited from traveling there. The Israel Defense Forces (IDF) are conducting large-scale military operations in Gaza against Hamas, a U.S. government-designated foreign terrorist organization, which was responsible for the October 7 attack on Israel. As a result of the armed conflict, the security environment within Gaza and on its borders is extremely dangerous and volatile. The pedestrian crossing between Gaza and Israel was damaged on October 7 and remains closed, and the pedestrian crossing between Egypt and Gaza may close without advance notice depending on the security situation. There are sporadic telecommunication and internet outages within Gaza further inhibiting the ability of residents to obtain informa

In [4]:
reddit = pd.read_csv("reddit.csv")

reddit_pc = reddit[['title','selftext','url','creation_date','comments']].dropna()
reddit_pc.shape

(10677, 5)

In [5]:
reddit_pc['comments'] = reddit_pc['comments'].apply(lambda x:x.replace('[','').replace(']',''))

In [6]:
reddit_pc = reddit_pc.reset_index()

In [7]:
reddit_pc['token'] = reddit_pc['comments'].apply(lambda x:len(x.split(' ')))

In [8]:
reddit_pc = reddit_pc.loc[reddit_pc['token'] > 100]
reddit_pc.shape

(9450, 7)

In [9]:
from nltk.tokenize import sent_tokenize

In [10]:
reddit_pc['split'] = reddit_pc['comments'].apply(lambda x: sent_tokenize(x))

In [11]:
reddit_pc['selftext'].nunique()

9424

In [12]:
from sentence_transformers import SentenceTransformer, util

rr_model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')


In [13]:
reddit_pc = reddit_pc.explode('split')
reddit_pc 

,index,title,selftext,url,creation_date,comments,token,split
0,0,Would I be banned by an airline for skiplagging?,My wife and I have a flight from Quito to Vanc...,https://www.reddit.com/r/travel/comments/17tnw...,2023-11-12 08:35:29,I live in Vancouver as well and booking a flig...,996,I live in Vancouver as well and booking a flig...
0,0,Would I be banned by an airline for skiplagging?,My wife and I have a flight from Quito to Vanc...,https://www.reddit.com/r/travel/comments/17tnw...,2023-11-12 08:35:29,I live in Vancouver as well and booking a flig...,996,"Annoying., Are you checking bags?"
0,0,Would I be banned by an airline for skiplagging?,My wife and I have a flight from Quito to Vanc...,https://www.reddit.com/r/travel/comments/17tnw...,2023-11-12 08:35:29,I live in Vancouver as well and booking a flig...,996,"If so, don't do this."
0,0,Would I be banned by an airline for skiplagging?,My wife and I have a flight from Quito to Vanc...,https://www.reddit.com/r/travel/comments/17tnw...,2023-11-12 08:35:29,I live in Vancouver as well and booking a flig...,996,"It'll cause all sorts of problems for you, the..."
0,0,Would I be banned by an airline for skiplagging?,My wife and I have a flight from Quito to Vanc...,https://www.reddit.com/r/travel/comments/17tnw...,2023-11-12 08:35:29,I live in Vancouver as well and booking a flig...,996,They'll be happy to use your seat for standbye...
...,...,...,...,...,...,...,...,...
10676,16286,Walkabout ideas?,Recent life events have left me needing a soci...,https://www.reddit.com/r/adventures/comments/1...,2023-01-22 07:34:05,I'm 42 and enjoy solo backcountry hiking as we...,258,Carvers gap to 19E is around 20 miles and capt...
10676,16286,Walkabout ideas?,Recent life events have left me needing a soci...,https://www.reddit.com/r/adventures/comments/1...,2023-01-22 07:34:05,I'm 42 and enjoy solo backcountry hiking as we...,258,"Then there's Grayson Highlands, specifically M..."
10676,16286,Walkabout ideas?,Recent life events have left me needing a soci...,https://www.reddit.com/r/adventures/comments/1...,2023-01-22 07:34:05,I'm 42 and enjoy solo backcountry hiking as we...,258,It's also around 20 miles.
10676,16286,Walkabout ideas?,Recent life events have left me needing a soci...,https://www.reddit.com/r/adventures/comments/1...,2023-01-22 07:34:05,I'm 42 and enjoy solo backcountry hiking as we...,258,I've also hiked the smokies section of the AT ...


In [46]:
for i,r in tqdm(reddit_pc[58718:].iterrows()):
    q_emb = rr_model.encode(r['title'] + ' ' + r['selftext'])
    com_emb = rr_model.encode(r['split'])
    question_split_scores = util.dot_score([q_emb],[com_emb])[0].cpu().tolist()
    # print(question_dot_scores)
    reddit_pc.loc[i,'dot'] = question_split_scores[0]
    
    

67it [00:04, 17.64it/s]

In [26]:
reddit_pc.dropna().shape

(58717, 9)

In [27]:
def get_max_score(group):
    return group.loc[group['dot'].idxmax()]

In [41]:
reddit_pc_no_nulls = reddit_pc.dropna()
reddit_pc_no_nulls_cleaned = reddit_pc_no_nulls.loc[reddit_pc_no_nulls['dot'] > 0.6]

reddit_pc_no_nulls_cleaned_grouped = reddit_pc_no_nulls_cleaned.groupby('selftext').apply(get_max_score).drop_duplicates()
# reddit_pc_no_nulls

In [42]:
reddit_pc_no_nulls_cleaned_grouped.shape

(2095, 9)

In [45]:
reddit_pc_no_nulls_cleaned_grouped['selftext'].nunique()

78

In [36]:
index_name = 'wanderchat-reddit-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of ada 002
        metric='cosine',
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [427]:
data = reddit_pc_no_nulls_cleaned_grouped  # this makes it easier to iterate over the dataset

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    try:
        i_end = min(len(data), i+batch_size)
        # get batch of data
        batch = data.iloc[i:i_end]
        # generate unique ids for each chunk
        ids = [f"{x['index']}" for i, x in batch.iterrows()]
        # get text to embed
        texts = [x['split'] for _, x in batch.iterrows()]
        # embed text
        embeds = embed_model.embed_documents(texts)
        # get metadata to store in Pinecone
        metadata = [
            {'text': x['split'],
            'source': x['url'],
            'title': x['title'],
            'question':x['selftext'],
            'creation_date':x['creation_date']} for i, x in batch.iterrows()
        ]
        # add to Pinecone
        index.upsert(vectors=zip(ids, embeds, metadata))
    except:
        pass

  0%|          | 7/3964 [00:41<6:34:49,  5.99s/it]